In [1]:
import os
import sys
import csv
import pickle
import random
import numpy as np
from time import time
from tqdm import tqdm

from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchtext.vocab import build_vocab_from_iterator

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
# def read_data(path):
#     with open(path, 'r') as csvfile:
#         train_data = list(csv.reader(csvfile))[1:] # skip col name
#         sents, lbls = [], []
#         for i in range(0, len(train_data), 16):
#             s, l = zip(*train_data[i:i+16])
#             sents.append(s)
#             lbls.append(l)
#     return sents, lbls

def read_data(path):
    with open(path, 'r') as csvfile:
        train_data = list(csv.reader(csvfile))[1:] # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls

# number of trainable parameters in model
def get_total_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl
    
    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]
    
    def __len__(self):
        return len(self.lbl)
    
class CleavageBatch:
    def __init__(self, batch: List[Tuple[str, str]]):
        ordered_batch = list(zip(*batch))
        self.seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.float)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def collate_wrapper(batch):
    return CleavageBatch(batch)

In [5]:
class Attention(nn.Module):
    def __init__(self, input_dim, hidden):
        super().__init__()

        self.W0 = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(input_dim, hidden)).to(device)
        )
        self.W = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(hidden, 1)).to(device)
        )
        self.b0 = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(hidden, 1)).squeeze().to(device)
        )
        self.b = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(1, 1)).squeeze(1).to(device)
        )

    def forward(self, x):
        # input shape: (batch_size, num_filters, seq_len)
        # input needs to be permuted
        energy = (
            x.permute(0, 2, 1) @ self.W0 + self.b0
        )  # linear activation, i.e. only identity
        # print('energy1: ', energy.shape)

        # input shape: (batch_size, seq_len, hidden)
        energy = (energy @ self.W + self.b).squeeze()
        # print('energy2: ', energy.shape)

        # input shape: (batch_size, seq_len)
        energy = F.softmax(energy, dim=1)
        # print('after softmax: ', energy.shape)

        # input shape: energy=(batch_size, seq_len)
        # output shape: (batch_size, input_dim)
        # batch-wise dot product along dims 1
        res = (energy.unsqueeze(1) @ x.permute(0, 2, 1)).squeeze(1)
        # print('batch dot: ', res.shape)

        # input shape: (batch_size, input_dim)
        # output shape: (batch_size, input_dim+seq_len)
        return torch.concat([res, energy], axis=-1)

In [11]:
class CNNAttention(nn.Module):
    def __init__(
        self,
        seq_len,
        num_filters1,
        kernel_size1,
        num_filters2,
        kernel_size2a,
        kernel_size2b,
        kernel_size2c,
        num_filters3,
        kernel_size3a,
        kernel_size3b,
        kernel_size3c,
        attention_hidden1,
        attention_hidden2,
        hidden_size1,
        hidden_size2,
        dropout1,
        dropout2,
        dropout3,
    ):
        super().__init__()

        self.dropout1 = nn.Dropout(dropout1)
        self.dropout2 = nn.Dropout(dropout2)
        self.dropout3 = nn.Dropout(dropout3)

        self.conv1 = nn.Conv1d(
            in_channels=1,
            out_channels=num_filters1,  # 200
            kernel_size=kernel_size1,  # 1
            padding="same",
        )

        self.conv2a = nn.Conv1d(
            in_channels=num_filters1,  # 200
            out_channels=num_filters2,  # 150
            kernel_size=kernel_size2a,  # 9
            padding="same",
        )

        self.conv2b = nn.Conv1d(
            in_channels=num_filters1,  # 200
            out_channels=num_filters2,  # 150
            kernel_size=kernel_size2b,  # 6
            padding="same",
        )

        self.conv2c = nn.Conv1d(
            in_channels=num_filters1,  # 200
            out_channels=num_filters2,  # 150
            kernel_size=kernel_size2c,  # 3
            padding="same",
        )

        self.conv3a = nn.Conv1d(
            in_channels=3 * num_filters2,
            out_channels=num_filters3,  # 200
            kernel_size=kernel_size3a,  # 10?
            padding="same",
        )

        self.conv3b = nn.Conv1d(
            in_channels=3 * num_filters2,
            out_channels=num_filters3,  # 200
            kernel_size=kernel_size3b,  # 5
            padding="same",
        )

        self.conv3c = nn.Conv1d(
            in_channels=3 * num_filters2,
            out_channels=num_filters3,  # 200
            kernel_size=kernel_size3c,  # 15
            padding="same",
        )

        self.attention1 = Attention(input_dim=num_filters3, hidden=attention_hidden1)
        self.attention1r = Attention(input_dim=seq_len, hidden=attention_hidden2)

        self.attention2 = Attention(input_dim=num_filters3, hidden=attention_hidden1)
        self.attention2r = Attention(input_dim=seq_len, hidden=attention_hidden2)

        self.attention3 = Attention(input_dim=num_filters3, hidden=attention_hidden1)
        self.attention3r = Attention(input_dim=seq_len, hidden=attention_hidden2)

        self.fc1 = nn.Linear(3*num_filters3 + 3*seq_len, hidden_size1)  # 149
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)  # 8
        self.fc3 = nn.Linear(hidden_size2, 1)

    def forward(self, seq):
        ### encoder ###

        # in: (batch_size, seq_len), out: (batch_size, num_filters1, seq_len)
        # print("inp: ", seq.unsqueeze(1).shape)
        x1 = F.leaky_relu(self.dropout1(self.conv1(seq.unsqueeze(1))))  # add dim=1 for conv
        # print("x1: ", x1.shape)

        # in: (batch_size, num_filters1, seq_len)
        # out: (batch_size, num_filters2, seq_len)
        y1 = F.leaky_relu(self.dropout1(self.conv2a(x1)))
        # print("y1: ", y1.shape)
        y2 = F.leaky_relu(self.dropout1(self.conv2b(x1)))
        # print("y2: ", y2.shape)
        y3 = F.leaky_relu(self.dropout1(self.conv2c(x1)))
        # print("y3: ", y3.shape)
        # cat_y out: (batch_size, 3*num_filters2, seq_len)
        cat_y = self.dropout1(torch.cat([y1, y2, y3], dim=1))
        # print("mergeY: ", res_y.shape)

        # in: (batch_size, 3*num_filters2, seq_len)
        # out: (batch_size, num_filters3, seq_len)
        z1 = self.dropout1(F.leaky_relu(self.conv3a(cat_y)))
        # print("z1: ", z1.shape)
        z2 = self.dropout1(F.leaky_relu(self.conv3b(cat_y)))
        # print("z2: ", z2.shape)
        z3 = self.dropout1(F.leaky_relu(self.conv3c(cat_y)))
        # print("z3: ", z3.shape)

        ### decoder ###

        # in: (batch_size, num_filters3, seq_len)
        # out w/out r, e.g. decoded_z1: (batch_size, num_filters3)
        # out w/ r: e.g. decoded_z1r: (batch_size, seq_len)
        decoded_z1 = self.attention1(z1)[:, : z1.shape[1]]
        # print("decoded_z1: ", decoded_z1.shape)
        decoded_z1r = self.attention1r(z1.permute(0, 2, 1))[:, : z1.shape[2]]
        # print("decoded_z1r: ", decoded_z1r.shape)

        decoded_z2 = self.attention2(z2)[:, : z2.shape[1]]
        # print("decoded_z2: ", decoded_z2.shape)
        decoded_z2r = self.attention2r(z2.permute(0, 2, 1))[:, : z2.shape[2]]
        # print("decoded_z2r: ", decoded_z2r.shape)

        decoded_z3 = self.attention3(z3)[:, : z3.shape[1]]
        # print("decoded_z3: ", decoded_z3.shape)
        decoded_z3r = self.attention3r(z3.permute(0, 2, 1))[:, : z3.shape[2]]
        # print("decoded_z3r: ", decoded_z3r.shape)

        # out: (batch_size, 3*num_filters3 + 3*seq_len)
        cat = self.dropout2(
            torch.cat(
                [
                    decoded_z1,
                    decoded_z1r,
                    decoded_z2,
                    decoded_z2r,
                    decoded_z3,
                    decoded_z3r,
                ],
                dim=-1,
            )
        )
        # print("output shape: ", cat.shape)

        # in: (batch_size, 3*num_filters3 + 3*seq_len), out: (batch_size, hidden_size1)
        out = self.dropout3(F.leaky_relu(self.fc1(cat)))
        # in: (batch_size, hidden_size1), out: (batch_size, hidden_size2)
        out = self.dropout3(F.leaky_relu(self.fc2(out)))
        # in: (batch_size, hidden_size2), out: (batch_size)
        return self.fc3(out).squeeze()

In [12]:
def process(model, loader, criterion, optim=None):
    epoch_loss, num_correct, total = 0, 0, 0
    preds = []
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores = model(seq)
        loss = criterion(scores, lbl)
        
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        
        preds.append(scores > 0)
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum()
        total += len(seq)
    return epoch_loss / total, num_correct / total, preds

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# load train and dev data
train_seqs, train_lbl = read_data('../../data/n_train.csv')
dev_seqs, dev_lbl = read_data('../../data/n_val.csv')

# create vocab from train seqs
vocab = build_vocab_from_iterator(train_seqs, specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])
encode_text = lambda x: vocab(list(x))

In [13]:
NUM_EPOCHS = 10
BATCH_SIZE = 512
VOCAB_SIZE = len(vocab)
LEARNING_RATE = 3e-4


params = {
    'seq_len': 10,
    'num_filters1': 200,
    'kernel_size1': 1,
    'num_filters2': 150,
    'kernel_size2a': 9,
    'kernel_size2b': 5,
    'kernel_size2c': 3,
    'num_filters3': 200,
    'kernel_size3a': 9,
    'kernel_size3b': 5,
    'kernel_size3c': 15,
    'attention_hidden1': 10,
    'attention_hidden2': 8,
    'hidden_size1': 149,
    'hidden_size2': 8,
    'dropout1': 0.,
    'dropout2': 0.,
    'dropout3': 0.
}

model = CNNAttention(**params).to(device)


optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

# create train and dev loader
train_data = CleavageDataset(train_seqs, train_lbl)
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

dev_data = CleavageDataset(dev_seqs, dev_lbl)
dev_loader = DataLoader(dev_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

print(f"Total trainable model parameters: {get_total_model_params(model):,}")

Total trainable model parameters: 3,223,032


In [14]:
start = time()
print("Starting Training.")
highest_val_acc = 0
train_losses, train_accuracies= [], []
val_losses, val_accuracies = [], []

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss, train_acc, train_preds = process(model, train_loader, criterion, optimizer)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_acc, val_preds = process(model, dev_loader, criterion)
        
    # save current acc, loss
    train_losses.append((epoch, train_loss))
    train_accuracies.append((epoch, train_acc))
    val_losses.append((epoch, val_loss))
    val_accuracies.append((epoch, val_acc))
    
    if val_acc > highest_val_acc:
        highest_val_acc = val_acc
        path = f"../../params/n_term/quadBiLSTM/acc{val_acc:.4f}_epoch{epoch}.pt"
        torch.save(model.state_dict(), path)
        
    print(
        f"Training:   [Epoch {epoch:2d}, Loss: {train_loss:8.4f}, Acc: {train_acc:.4f}]"
    )
    print(f"Evaluation: [Epoch {epoch:2d}, Loss: {val_loss:8.4f}, Acc: {val_acc:.4f}]")
    
print("Finished Training.")
train_time = (time() - start) / 60
print(f"Training took {train_time} minutes.")

Starting Training.
Eval: 100%|██████████████████████████████████████████████████████| 280/280 [00:00<00:00, 307.20batches/s]
Training:   [Epoch  1, Loss:   0.0013, Acc: 0.6144]
Evaluation: [Epoch  1, Loss:   0.0012, Acc: 0.6487]
Eval: 100%|██████████████████████████████████████████████████████| 280/280 [00:00<00:00, 306.60batches/s]
Training:   [Epoch  2, Loss:   0.0012, Acc: 0.6571]
Evaluation: [Epoch  2, Loss:   0.0012, Acc: 0.6552]
Eval: 100%|██████████████████████████████████████████████████████| 280/280 [00:00<00:00, 308.59batches/s]
Training:   [Epoch  3, Loss:   0.0012, Acc: 0.6629]
Evaluation: [Epoch  3, Loss:   0.0012, Acc: 0.6620]
Eval: 100%|██████████████████████████████████████████████████████| 280/280 [00:00<00:00, 306.09batches/s]
Training:   [Epoch  4, Loss:   0.0012, Acc: 0.6659]
Evaluation: [Epoch  4, Loss:   0.0012, Acc: 0.6671]
Eval: 100%|██████████████████████████████████████████████████████| 280/280 [00:00<00:00, 312.37batches/s]
Training:   [Epoch  5, Loss:   0.00

In [ ]:
# save training stats
lsts = [train_losses, train_accuracies, val_losses, val_accuracies, train_time]
names = [
    "train_losses",
    "train_accuracies",
    "val_losses",
    "val_accuracies",
    "train_time",
]
to_save = dict()
for name, lst in zip(names, lsts):
    to_save[name] = lst

with open(f"../params/n_term/quadBiLSTM/metrics.pkl", "wb") as f:
    pickle.dump(to_save, f, pickle.HIGHEST_PROTOCOL)

print("Finished Saving Details.")